In [ ]:
import datetime as dt
from gas_stor_opt.gas_storage_optim.gs_optim import GasStorage
from gas_stor_opt.snowflake_conn import (
    download_curve_from_snowflake,
    establish_snowflake_connector,
    prepare_curve_from_snowflake,
)
from gas_stor_opt.utils import (
    check_for_duplicate_storage_names,
    import_prices,
    initialize_storage,
)
import json as json
import numpy as np
import pandas as pd

In [ ]:
import snowflake.connector

In [ ]:
from typing import Optional
import datetime
import gas_stor_opt.config as config
import pandas as pd
import pytz
import snowflake.connector


def establish_snowflake_connector(
    account: Optional[str] = config.SNFL_ACCOUNT,
    user: Optional[str] = config.SNFL_USER,
    password: Optional[str] = config.SNFL_PASSWORD,
) -> snowflake.connector.SnowflakeConnection:
    snowflake_connector = snowflake.connector.connect(
        account=account, user=user, password=password
    )
    snowflake_connector.cursor()
    return snowflake_connector


def download_curve_from_snowflake(
    snowflake_connector: snowflake.connector.SnowflakeConnection,
    price_date: datetime.date,
    price_curve_id: Optional[str] = "GAS_EON_CZVTP_PFC_EOD_D_MID_FINAL",
) -> pd.DataFrame:
    query = f"""
    SELECT TIME, VALUE, PRICEDATE from
    "Q_EEDP"."DP_PRICE"."PRICE_SERIES"
    WHERE PRICECURVEID = '{price_curve_id}' and
    PRICEDATE = CAST(DATEFROMPARTS({price_date.year}, {price_date.month}, {price_date.day}) as DATE)
    ORDER BY PRICEDATE DESC, TIME ASC
    """
    curve = pd.read_sql_query(query, con=snowflake_connector)
    return curve


def prepare_curve_from_snowflake(raw_snowflake_table: pd.DataFrame) -> pd.DataFrame:
    curve = raw_snowflake_table.rename(columns={"VALUE": "price", "TIME": "date"})[
        ["date", "price"]
    ]
    curve["date"] = pd.to_datetime(curve["date"], utc=True).dt.tz_convert(
        tz=pytz.timezone("Europe/Prague")
    )
    curve = (
        curve.set_index(keys="date", drop=True, inplace=False)
        .resample(rule="MS")
        .mean()
        .reset_index()
    )
    curve["date"] = curve["date"].apply(lambda x: x.replace(tzinfo=None))
    return curve

In [ ]:
import gas_stor_opt.config as config

In [ ]:
import os


SNFL_ACCOUNT = os.getenv("SNFL_ACCOUNT")
SNFL_USER = os.getenv("SNFL_USER")
SNFL_PASSWORD = os.getenv("SNFL_PASSWORD")
account = config.SNFL_ACCOUNT
user = config.SNFL_USER
password = config.SNFL_PASSWORD

In [ ]:
snowflake_connector = snowflake.connector.connect(
    account=account, user=user, password=password
)
snowflake_connector.cursor()

In [ ]:
snowflake_connector = establish_snowflake_connector()

In [ ]:
prices = import_prices(uploaded_file="../gas_stor_opt/data/prices1.xlsx")
with open("../gas_stor_opt/data/storages.json", "r") as file:
    # with open("app/src/data/storages.json", "r") as file:  # for Docker build
    storages_json = json.load(file)
storage_json = storages_json[6]

In [ ]:
storage_json

In [ ]:
stor_name = "pokus"
date_start = dt.date(2024, 4, 1)
date_end = dt.date(2025, 3, 31)
initial_state = 0
empty_on_end_date = True
optimization_time_limit = None

In [ ]:
stor_name = "pokus"
date_start = dt.date(2024, 4, 1)
date_end = dt.date(2025, 3, 31)
initial_state = 0
empty_on_end_date = True
optimization_time_limit = None

storage = GasStorage(name=stor_name, date_start=date_start, date_end=date_end)
storage.load_prices(prices)
for period in storage_json["TimePeriods"]:
    period_start_date = dt.datetime.strptime(period["StartDate"], "%Y-%m-%d").date()
    period_end_date = dt.datetime.strptime(period["EndDate"], "%Y-%m-%d").date()
    if period_end_date >= date_start:
        storage.load_attribute("wgv", period["WGV"], period_start_date, period_end_date)
        storage.load_attribute(
            "wr", period["WithdrawalRate"], period_start_date, period_end_date
        )
        storage.load_attribute(
            "ir", period["InjectionRate"], period_start_date, period_end_date
        )
storage.load_attribute(
    "inj_curve",
    np.array(storage_json["InjectionCurve"]) / 100,
    date_start,
    date_end,
)
storage.load_attribute(
    "wit_curve",
    np.array(storage_json["WithdrawalCurve"]) / 100,
    date_start,
    date_end,
)
storage.set_initial_state(initial_state)
storage.set_injection_season(storage_json["InjectionSeason"])
storage.set_state_to_date(
    {int(key): val for key, val in storage_json["StatesToDate"].items()}
)
dates_to_empty_storage = []
for date in storage_json["DatesToEmptyStorage"]:
    dates_to_empty_storage.append(dt.datetime.strptime(date, "%Y-%m-%d").date())
if empty_on_end_date:
    dates_to_empty_storage.append(date_end)
storage.set_dates_to_empty_storage(dates_to_empty_storage)
if optimization_time_limit is not None:
    storage.set_optimization_time_limit(optimization_time_limit)
else:
    storage.set_optimization_time_limit(storage_json["DefaultTimeLimit"])

storage.create_model()
storage.solve_model(solver_name="scip", stream_solver=True)

In [ ]:
storage.attr

In [ ]:
excel_file = pd.read_excel(
    "./gas_stor_opt/data/prices1.xlsx", usecols=["date", "price"]
)

In [ ]:
excel_file = pd.read_excel(
    "./gas_stor_opt/data/prices1.xlsx",
    parse_dates=True,
    usecols="A:B",
    names=["date", "price"],
)

In [ ]:
excel_file = pd.read_excel(
    "./gas_stor_opt/data/zasobniky.xlsx", usecols="A:B", names=["date", "price"]
)

In [ ]:
excel_file = pd.read_excel(
    "./gas_stor_opt/data/prices1.xlsx", usecols="A:B", names=["date", "price"]
)
dates = pd.to_datetime(excel_file["date"], errors="coerce", format="%d-%m-%Y")
excel_file["date"] = dates

In [ ]:
def load_prices(imported_prices):
    prices_monthly = imported_prices[
        imported_prices["date"] >= pd.to_datetime(dt.date(2024, 2, 1).replace(day=1))
    ]
    prices_monthly["year"] = prices_monthly.loc[:, "date"].dt.year
    prices_monthly["month"] = prices_monthly.loc[:, "date"].dt.month
    # attr["prices"] = (
    #     pd.merge(attr, prices_monthly, on=["year", "month"])
    #     .sort_values(["year", "month"])["price"]
    #     .values
    # )
    return prices_monthly

In [ ]:
load_prices(imported_prices=excel_file)

In [ ]:
import datetime as dt


dt.date.today()

In [ ]:
from msal_streamlit_authentication import msal_authentication


msal_authentication

In [ ]:
import json


storages_json = json.load(open("./gas_stor_opt/data/storages.json"))

dt.datetime.strptime(
    storages_json["RWE"]["TimePeriods"][0]["EndDate"], "%Y-%m-%d"
).date()

In [ ]:
storages_json["RWE"]

In [ ]:
for i in storages_json:
    print(i)

In [ ]:
len(storages_json[1]["TimePeriods"])

In [ ]:
storages_json[1]["TimePeriods"][1]["EndDate"]

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient


# Acquire a credential object
credential = DefaultAzureCredential(exclude_environment_credential=True)

blob_service_client = BlobServiceClient(
    account_url="https://datqadl01.blob.core.windows.net", credential=credential
)

# Get a client to interact with the container
container_name = "data-lake"
container_client = blob_service_client.get_container_client(container_name)

blob_client = container_client.get_blob_client(
    "curated/business-managed/cz/portfolio-management/storages.json"
)
with open("jsonfile.json", "wb") as my_blob:
    download_stream = blob_client.download_blob()
    my_blob.write(download_stream.readall())

In [ ]:
# Get a client to interact with the container
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
from azure.identity import ClientSecretCredential
from azure.storage.blob import BlobServiceClient


DefaultAzureCredential()

In [ ]:
from azure.identity import ClientSecretCredential
from azure.storage.blob import BlobServiceClient


# Azure AD service principal details
tenant_id = "XXX"
client_id = "XXX"
client_secret = "XXX"

# Azure Data Lake Storage details
storage_account_name = "datqadl01"
container_name = "data-lake"

# Authenticate using service principal
credential = ClientSecretCredential(
    tenant_id=tenant_id, client_id=client_id, client_secret=client_secret
)

# Create a BlobServiceClient
blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=credential,
)

# Get a client to interact with the container
container_client = blob_service_client.get_container_client(container_name)

# Now you can perform operations like uploading, downloading files from the container, etc.

blob_client = container_client.get_blob_client(
    "curated/business-managed/cz/portfolio-management/storages.json"
)
with open("jsonfile.json", "wb") as my_blob:
    download_stream = blob_client.download_blob()
    my_blob.write(download_stream.readall())

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient


storage_account_name = "datqadl01"
container_name = "data-lake"
token_credential = DefaultAzureCredential(token=login_token)

blob_service_client = BlobServiceClient(
    account_url=f"https://{storage_account_name}.blob.core.windows.net",
    credential=token_credential,
)
container_client = blob_service_client.get_container_client(container_name)
blob_client = container_client.get_blob_client(
    "curated/business-managed/cz/portfolio-management/storages.json"
)
# blob_client = container_client.get_blob_client("curated/gdm/eon/spot/de/pwr/2020/06/25/SpotPrice_PWR_DE_EPEXPeak_d_09_20_20200625.json")

with open("jsonfile.json", "wb") as my_blob:
    download_stream = blob_client.download_blob()
    my_blob.write(download_stream.readall())